# Gluon RNN Classifier for Human activity recognition

Classify human activity based on smartphone accelerometer and gyroscope data with gluon. A handy extensible RNNClassifier Gluon is provided which makes it simple to use RNN for other problems. 

## Dataset

Human Activity Recognition Using Smartphones Data Set 

https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones

The experiments have been carried out with a group of 30 volunteers within an age bracket of 19-48 years. Each person performed six activities (WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING) wearing a smartphone (Samsung Galaxy S II) on the waist. Using its embedded accelerometer and gyroscope, we captured 3-axial linear acceleration and 3-axial angular velocity at a constant rate of 50Hz. The experiments have been video-recorded to label the data manually. The obtained dataset has been randomly partitioned into two sets, where 70% of the volunteers was selected for generating the training data and 30% the test data. 

The sensor signals (accelerometer and gyroscope) were pre-processed by applying noise filters and then sampled in fixed-width sliding windows of 2.56 sec and 50% overlap (128 readings/window). The sensor acceleration signal, which has gravitational and body motion components, was separated using a Butterworth low-pass filter into body acceleration and gravity. The gravitational force is assumed to have only low frequency components, therefore a filter with 0.3 Hz cutoff frequency was used. From each window, a vector of features was obtained by calculating variables from the time and frequency domain.

## Overview

Follow this link to see a video of how the data was collected:

<p align="center">
  <a href="http://www.youtube.com/watch?feature=player_embedded&v=XOEN9W05_4A
" target="_blank"><img src="http://img.youtube.com/vi/XOEN9W05_4A/0.jpg" 
alt="Video of the experiment" width="400" height="300" border="10" /></a>
  <a href="https://youtu.be/XOEN9W05_4A"><center>[Watch video]</center></a>
</p>

In [3]:
# CODE
import mxnet
import numpy as np
from data_io_util import load_data, LABELS, INPUT_SIGNAL_TYPES
from mxnet_rnn_util import BaseRNNClassifier

 

## Data preparation

You can download the dataset and unzip it with its existing folder structure from https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones

** To make it easier, its been provided in the data/ folder **

In [4]:
### Load Data
train = ["data/har_data/train/%strain.txt" % signal for signal in INPUT_SIGNAL_TYPES]
test = ["data/har_data/test/%stest.txt" % signal for signal in INPUT_SIGNAL_TYPES]


X_train = load_data(train)
X_test = load_data(test)

X_train.shape

(7352, 128, 9)

### Load training labels

In [5]:
import pandas as pd

y_train_path = "data/har_data/train/y_train.txt"

df = pd.read_csv(y_train_path, names=["labels"])
y_train = np.asarray(list(df.to_dict()['labels'].values())).astype('float32')

y_test_path = "data/har_data/test/y_test.txt"
df = pd.read_csv(y_test_path, names=["labels"])
y_test = np.asarray(list(df.to_dict()['labels'].values())).astype('float32')

y_train.shape, y_test.shape

((7352,), (2947,))

### A Gluon RNN Helper class has been provided here to help extend and reuse the same code for other problems


### Network definition and training

In [ ]:
ctx = mxnet.cpu() # .gpu(0) #change context to execute on CPU
model = BaseRNNClassifier(ctx)
model.build_model(n_out=len(LABELS), rnn_size=64, n_layer=1)
model.compile_model()

In [ ]:
train_loss, train_acc, test_acc = model.fit([X_train, y_train], [X_test, y_test], batch_size=32, epochs=25)

#### Lets plot accuracy and loss over each epoch

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(train_loss, label="loss")
plt.plot(train_acc, label="train")
plt.plot(test_acc, label="validation")
plt.legend()
plt.show()

In [ ]:
# Loss plot

plt.plot(train_loss, label="loss")
plt.legend()
plt.show()

In [ ]:
!mkdir models
model.save_parameters('models/model.params')

## Excercise 

1. Build the confusion matrix 
2. Try plotting the misclassified samples
3. Can we use a CNN to solve this problem? 